https://github.com/monologg/KoELECTRA

In [104]:
# import library
import pandas as pd
from tqdm import tqdm
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TextClassificationPipeline

In [83]:
ice = pd.read_csv('data/icecream_emot.csv')
tsh = pd.read_csv('data/sherpa_emot3.csv')

In [84]:
tsh = tsh[['title_comments', 'star']]
tsh.rename(columns={'title_comments' : 'text', 'star' : 'score'}, inplace=True)
tsh['score'] = tsh['score'].apply(lambda x : int(x.replace('점','')))
tsh['score'] = tsh['score'].apply(lambda x : 0 if x == 3 else (1 if x > 3 else -1))
tsh

,text,score
0,법정연수 유용하네요 의무적으로 해마다 듣는 연수인데 다른 강사에게 듣고 싶어 신청했...,1
1,"독립운동에 관한 지식 쌓기 독립운동에 대해서 알고, 학생 대상으로 수업하기 좋았습니...",1
2,삶을 바꾸는 사회수업 주제통합 질문과 상상으로 진행하기 사회 교과서 개정되고 학습량...,1
3,영어 회화가 막막한 선생님들께 유익한 연수! 영어가 너무 오랜만이시거나 일상생활에서...,1
4,독서교육을 해보고 싶어지게 만드는 연수!!! 아이를 키우며 그림책과 동화책에 관심을...,1
...,...,...
2595,알찬연수 고맙습니다.^^ 학교 진로교육의 목표 (2015개정)학생 자신의 진로를 창...,1
2596,"스토리텔링을 활용한 초등수학 수업 연수 굿굿! 선생님들의 수학 개념 설명을 듣고, ...",1
2597,고맙습니다. 진로라는게 흥미는 있지만 너무 포괄적이네요.,1
2598,감사합니다^^ 진로교육에 관해 많은 것을 알게 되었습니다.\n감사합니다.,1


In [85]:
ice = ice[['title_comments', 'rated']]
ice.rename(columns={'title_comments' : 'text', 'rated' : 'score'}, inplace=True)
ice['score'] = ice['score'].apply(lambda x : int(x[-1]))
ice['score'] = ice['score'].apply(lambda x : 0 if x == 3 else (1 if x > 3 else -1))
ice

,text,score
0,알차네요 무엇보다도 빨리들을 수 있어 좋습니다,1
1,명불허전 명강사라 그런지 말씀을 참 잘하시네요~,1
2,강추 30년 가까이 일하면서도 놓치고 있었던 것들을 알려주었습니다. 그리고 무조건 ...,1
3,패들렛과 띵커벨 활용 잘 할 수 있어요 패들렛과 띵커벨 활용하고 있었으나 다양한 교...,1
4,"필수 연수 부담 없이 편안하게 안전 연수 올 해 꼭 받아야 하는데, 부담 없이 편안...",1
...,...,...
5236,알베르토가 함께하는 핵심역량 연계 다문화 교육 여타 연수와 다르게 새로 알게된 내용...,1
5237,전문가가 알려주는 창의 영재교육의 비법 연수 후기 영재 교육에 대한 틀을 잡아주시는...,1
5238,도도한 교사생활 연수를 통해 자신감을 더 갖게 되었다.,1
5239,교실에서 활용할 수 있는 앱들 중심으로 재미있게 들었어요. 교실에서 활용할 수 있는...,1


tsh modeling

In [233]:

model_lst = ['jaehyeong/koelectra-base-v3-generalized-sentiment-analysis', 
            'klue/bert-base', 
            'matthewburke/korean_sentiment',
            # 'monologg/koelectra-base-discriminator',
            # "monologg/koelectra-small-discriminator",
            "monologg/koelectra-base-v2-discriminator",
            # "monologg/koelectra-small-v2-discriminator",
            # "monologg/koelectra-base-v3-discriminator",
            # "monologg/koelectra-small-v3-discriminator",
            # "facebook/bart-large-mnli",
            # "monologg/koelectra-base-v3-goemotions",
            'skt/kogpt2-base-v2',
            # 'skt/kobert-base-v1',
            'beomi/kcbert-large',
            # "google/electra-small-discriminator"
            ]


for i in model_lst[1:]: # 0,1,2,3
    # load model
    pre_model = i
    tokenizer = AutoTokenizer.from_pretrained(pre_model)
    model = AutoModelForSequenceClassification.from_pretrained(pre_model)
    sentiment_classifier = TextClassificationPipeline(tokenizer=tokenizer, model=model)

    print('====='+ '\033[95m' + pre_model + '\033[0m' + '=====')

    # target reviews
    review_list = list(ice.text)

    ice[pre_model] = '-'
    # predict
    for idx, review in tqdm(enumerate(review_list)):
        pred = sentiment_classifier(review)
        # print(f'{review}\n>> {pred[0]}')
        if (pred[0]['label']=='1')&(pred[0]['score'] > 0.51) | (pred[0]['label']=='LABEL_1')&(pred[0]['score'] > 0.51):
            ice[pre_model][idx] = '1'
        elif (pred[0]['label']=='0')&(pred[0]['score']>0.51) | (pred[0]['label']=='LABEL_0')&(pred[0]['score'] > 0.51):  
            ice[pre_model][idx] = '-1'
        else:
            ice[pre_model][idx] = '0'

        # print(f'{review}\n>> {pred[0]}')

print('🥕')
print('')


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at klue/bert-base and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


=====klue/bert-base=====


0it [00:00, ?it/s]C:\Users\hslio\AppData\Local\Temp\ipykernel_9540\1665998401.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ice[pre_model][idx] = '1'
9it [00:00, 21.84it/s]C:\Users\hslio\AppData\Local\Temp\ipykernel_9540\1665998401.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ice[pre_model][idx] = '-1'
C:\Users\hslio\AppData\Local\Temp\ipykernel_9540\1665998401.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ice[pre

KeyboardInterrupt: 

In [227]:
for i in ice.iloc[:, 1:].columns:
    print(ice[[i]].value_counts())
    print()

score
 1       5146
 0         64
-1         31
dtype: int64

jaehyeong/koelectra-base-v3-generalized-sentiment-analysis
1                                                             5135
-1                                                             106
dtype: int64

matthewburke/korean_sentiment
0                                5241
dtype: int64

klue/bert-base
0                 5241
dtype: int64

monologg/koelectra-base-v2-discriminator
0                                           5241
dtype: int64



In [223]:
ice.iloc[:, 1:].columns

Index(['score', 'jaehyeong/koelectra-base-v3-generalized-sentiment-analysis',
       'matthewburke/korean_sentiment', 'klue/bert-base',
       'monologg/koelectra-base-v2-discriminator'],
      dtype='object')